In [87]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
from scipy.spatial.distance import cosine
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util


In [31]:
!pip install transformers
nltk.download('stopwords')

In [4]:
df = pd.read_csv('data/billboard-lyrics-spotify.csv', sep=',', header=0)


In [5]:
df['year'] = pd.to_datetime(df['year'], format='%Y')

In [6]:
df['decade'] = df['year'].dt.year // 10 * 10

In [ ]:
df.dropna(subset=['lyrics'], inplace=True)


In [76]:
grouped_df = df.groupby('decade')

In [77]:
result = grouped_df['lyrics'].count()
result


decade
1960    975
1970    986
1980    994
1990    933
2000    919
2010    709
Name: lyrics, dtype: int64

In [80]:
grouped_df.get_group(2010)


,artist_all,artist_base,rank,song,year,artist_featured,song_clean,artist_clean,lyrics,acousticness,...,speechiness,tempo,time_signature,valence,duration_min,num_words,words_per_sec,num_uniq_words,decade,uniq_ratio
4850,kesha,kesha,1,tik tok,2010-01-01,NaN,tik tok,kesha,grab my glasses im out the door im gonna hit ...,0.07460,...,0.1160,120.032,4.0,0.735,3.328217,458.0,2.293521,136.0,2010,3.367647
4851,lady antebellum,lady antebellum,2,need you now,2010-01-01,NaN,need you now,lady antebellum,picture perfect memories scattered all around...,0.09270,...,0.0303,107.943,4.0,0.231,4.626217,215.0,0.774571,77.0,2010,2.792208
4852,train,train,3,"hey, soul sister",2010-01-01,NaN,hey soul sister,train,heyy heeey heey your lipstick stains on the f...,0.21700,...,0.0436,97.030,4.0,0.768,3.611117,284.0,1.310767,115.0,2010,2.469565
4853,katy perry featuring snoop dogg,katy perry,4,california gurls,2010-01-01,snoop dogg,california gurls,katy perry,greetings loved ones lets take a journey i kn...,0.00446,...,0.0569,125.014,4.0,0.425,3.910883,445.0,1.896417,162.0,2010,2.746914
4854,usher featuring will.i.am,usher,5,omg,2010-01-01,will.i.am,omg,usher,oh my gosh baby let me i did it again so im g...,0.19800,...,0.0332,129.998,4.0,0.326,4.491550,526.0,1.951813,105.0,2010,5.009524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5561,camila cabello featuring young thug,camila cabello,96,havana,2017-01-01,young thug,havana,camila cabello,hey half of my heart is in havana ooh na na h...,0.18400,...,0.0300,104.988,4.0,0.394,3.621783,350.0,1.610625,102.0,2010,3.431373
5562,maroon 5 featuring sza,maroon 5,97,what lovers do,2017-01-01,sza,what lovers do,maroon 5,say say say hey hey now baby oh mama dont pla...,0.08050,...,0.0693,109.959,4.0,0.420,3.330817,366.0,1.831383,75.0,2010,4.880000
5563,blackbear,blackbear,98,do re mi,2017-01-01,NaN,do re mi,blackbear,do re mi fa so yeah yeah yeah oh do re mi fa ...,0.00419,...,0.0465,110.977,3.0,0.154,3.533783,382.0,1.801657,122.0,2010,3.131148
5564,xxxtentacion,xxxtentacion,99,look at me!,2017-01-01,NaN,look at me,xxxtentacion,ayy im like bitch who is your mans ayy cant k...,0.25900,...,0.2820,139.059,4.0,0.349,2.105767,337.0,2.667279,106.0,2010,3.179245


In [62]:
df.dropna(subset=['lyrics'], inplace=True)


In [103]:
grouped_lyrics = {}
for decade, group in grouped_df:
    lyrics = [item for item in group['lyrics']]
    grouped_lyrics[decade] = lyrics

In [81]:
# df['lyrics']
lyrics = [item for item in df['lyrics']]
lyrics_strings = list(map(str, lyrics))
lyrics_words = [word_tokenize(item) for item in lyrics_strings]

In [82]:
word_counts = [len(item) for item in lyrics_words]
avg_word_counts = np.mean(word_counts)
min_words = np.min(word_counts)
max_words = np.max(word_counts)
print('Average number of words per song: {}'.format(avg_word_counts))
print('Minimum number of words per song: {}'.format(min_words))
print('Maximum number of words per song: {}'.format(max_words))

Average number of words per song: 306.1104060913706
Minimum number of words per song: 0
Maximum number of words per song: 1153


In [7]:
sw = set(stopwords.words('english'))
lowercased = [[word.lower() for word in item] for item in lyrics_words]
removed_punctuation = [[word for word in review if word.isalnum()] for review in lowercased]
removed_stopwords = [[word for word in review if word not in sw] for review in removed_punctuation]

ps = PorterStemmer()
stemmed = [[ps.stem(word) for word in review] for review in removed_stopwords]

from nltk.util import ngrams

unigrams = [list(ngrams(item, 1)) for item in removed_punctuation]
bigrams = [list(ngrams(item, 2)) for item in removed_punctuation]
trigrams = [list(ngrams(item, 3)) for item in removed_punctuation]


In [88]:
# import BERT model

# load tokenizer (same one trained with the model)
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")

# load pre-trained BERT model weights
# model = BertModel.from_pretrained(
#     "unitary/toxic-bert",
#     output_hidden_states = True, # Whether the model returns all hidden-states.
# )

model = SentenceTransformer('unitary/toxic-bert')

# model = torch.hub.load('unitaryai/detoxify','multilingual_toxic_xlm_r')
# model.eval()



No sentence-transformers model found with name C:\Users\justi/.cache\torch\sentence_transformers\unitary_toxic-bert. Creating a new one with MEAN pooling.


In [106]:
decade_embeddings = {}

for decade in grouped_lyrics:
    # decade_embeddings[decade] = np.mean(group['embedding'].values, axis=0)
    # print(decade, group['lyrics'])
    # print(len(group['lyrics']))
    songs = grouped_lyrics[decade]
    decade_embeddings[decade] = model.encode(songs, convert_to_tensor=True)
    # for lyrics in group['lyrics']:
    #     tokens = tokenizer(lyrics, return_tensors='pt')
     


        

In [107]:
decade_mean = {}
for decade in decade_embeddings:
    decade_mean[decade] = torch.mean(decade_embeddings[decade], axis=0)

In [150]:
list(decade_mean.values())
array_decade_mean = torch.stack(list(decade_mean.values()))
np_means = array_decade_mean.numpy()

In [153]:
cossim = util.cos_sim(np_means, np_means)
cossim

tensor([[1.0000, 0.9991, 0.9980, 0.9755, 0.9149, 0.8693],
        [0.9991, 1.0000, 0.9984, 0.9793, 0.9237, 0.8792],
        [0.9980, 0.9984, 1.0000, 0.9817, 0.9230, 0.8815],
        [0.9755, 0.9793, 0.9817, 1.0000, 0.9769, 0.9530],
        [0.9149, 0.9237, 0.9230, 0.9769, 1.0000, 0.9921],
        [0.8693, 0.8792, 0.8815, 0.9530, 0.9921, 1.0000]])